In [1]:
import cv2
import numpy as np


In [2]:
from ultralytics import YOLO
# Burada kullanacağımız modeli seçiyoruz.
model= YOLO("yolov8x.pt") 

# Resim İçin Isı Haritası

In [7]:
# Görüntüyü yükle
image = cv2.imread('image.jpg')


# Burada görüntüyü RGB kanalına çevirip modele veriyoruz.
rgb_img=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
results = model(rgb_img,verbose=False) 
labels=results[0].names

# Nesnelerin orta noktasını tutmak için liste
points = []



for i in range(len(results[0].boxes)):
    x1,y1,x2,y2=results[0].boxes.xyxy[i]
    score=results[0].boxes.conf[i]
    # Bu hangi sınıfa ait olduğu. Bu değer sayı şeklinde olur.
    label=results[0].boxes.cls[i]
    x1,y1,x2,y2,score,label=int(x1),int(y1),int(x2),int(y2),float(score),int(label)
    # Bu sınıfın adı 
    name=labels[label]
    
    # Score'u bu değerin altında olan nesneleri almıyoruz.
    if score<0.1:
        continue
    # Sadece insan tespiti yapması için
    if name!='person':
        continue
    # orta noktaları bulup listeye ekliyoruz.
    center_x = int(x1/2+x2/2)
    center_y = int(y1/2+y2/2)
    points.append([center_x, center_y])
   
points = np.array(points)

# Isı haritasını oluşturuyoruz noktaları vererek. Buradaki bins değerini değiştirebilirsiniz.
# Bins değeri büyüdükçe ısı haritasında nesnelerin kapladığı yoğunluk daha az oluyor.
heatmap, xedges, yedges = np.histogram2d(points[:,1], points[:,0], bins=10, range=[[0, image.shape[0]], [0, image.shape[1]]])

# Bunu normalize ediyoruz.
heatmap = heatmap / np.max(heatmap) 

# Resmin boyutuyla eşleştiriyoruz
heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))

# Değerleri uygun formata getiriyoruz.
heatmap = np.uint8(255 * heatmap)

# bu değerlere göre ısı haritasını renklendiriyoruz.
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Sonuçları resimde göstermek için ısı haritası ile resmi topluyoruz.
# Burada ağırlıklı toplama var, orijinal resmin kuvveti 0.6, ısı haritasının ise 0.4
result = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)

# orijinal resim
cv2.imshow("kamera",image)

# ısı haritası
cv2.imshow('Isı Haritası', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Video İçin Isı Haritası

In [9]:
kamera= cv2.VideoCapture('video.mp4')
font = cv2.FONT_HERSHEY_SIMPLEX


while True:

   
    
    ret,kare=kamera.read()
    if not ret:
        break
    
    imgs=cv2.cvtColor(kare,cv2.COLOR_BGR2RGB)
    results = model(imgs,verbose=False)
    # burada her bir kare için noktalar sıfırlanır
    points = []
    labels=results[0].names
    for i in range(len(results[0].boxes)):
        x1,y1,x2,y2=results[0].boxes.xyxy[i]
        score=results[0].boxes.conf[i]
        label=results[0].boxes.cls[i]
        x1,y1,x2,y2,score,label=int(x1),int(y1),int(x2),int(y2),float(score),int(label)
        name=labels[label]
        
        if score<0.1:
            continue
        
        if name!='person':
            continue
        
        center_x = int(x1/2+x2/2)
        center_y = int(y1/2+y2/2)
        points.append([center_x, center_y])
     
    points = np.array(points)
    heatmap, xedges, yedges = np.histogram2d(points[:,1], points[:,0], bins=15, range=[[0, kare.shape[0]], [0, kare.shape[1]]])
    heatmap = cv2.resize(heatmap, (kare.shape[1], kare.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    
    result = cv2.addWeighted(kare, 0.6, heatmap, 0.8, 0)
    
   
    cv2.imshow('Isi Haritasi', result)

    cv2.imshow("kamera",kare)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
kamera.release()
cv2.destroyAllWindows()